# Solving the Van der Pol oscillator with the DeepXDE Framework
Raihaan Usman, UROP Student

DeepXDE is a data-driven framework developed by researchers at Brown University.

In [1]:
# surpress warnings from imports
import warnings
# warnings.filterwarnings("ignore")

import deepxde as dde
from deepxde.backend import tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import wandb

Using backend: tensorflow.compat.v1

2022-08-21 18:23:35.944589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-21 18:23:35.944624: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


2022-08-21 18:23:52.204913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-21 18:23:52.231069: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-21 18:23:52.231128: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-baf76a): /proc/driver/nvidia/version does not exist


In [2]:
sweep_config = {
  "name" : "optimisation-sweep",
  
  # Bayesian optimization of the parameters
  "method": "bayes",
  "metric": {"name": "val_loss", "goal": "minimize"},
  
  # "method" : "random",
  
  "parameters" : {
    
    "depth"       : {"values" : range(1, 11)},
    "height"      : {"values" : range(5, 55, 5)},
    "num_domain"  : {"values" : range(1000, 5000, 1000)}
    # "weight1" : {"min" : 0, "max" : 3},
    # "weight2" : {"min" : 0, "max" : 3},
    # "weight3" : {"min" : 0, "max" : 3}
    
  }
}

# "learning_rate" :{
#   "min": 0.0001,
#   "max": 0.1
# }

sweep_id = wandb.sweep(sweep_config, project="PINNs", entity="raihaan123")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: h5zmp3h3
Sweep URL: https://wandb.ai/raihaan123/PINNs/sweeps/h5zmp3h3


### Van der Pol Solver

In [ ]:
# PDE parameters
mu = 0.5
X0 = [1, 0]
t_f = 20

# Hyperparameters for the ADAM optimiser
# alpha           = 0.005
# loss_weights    = [3, 0.1, 0.1]    # [X, x1(0), x2(0)] errors

# VdP oscillator
ODE         = lambda X, t:  np.array([X[1], mu*(1-X[0]**2)*X[1]-X[0]])
true_sol    = lambda t:     odeint(ODE, X0, t)

# The PDE problem
def VdP(t, y):
    dy_dt = dde.grad.jacobian(y, t)
    d2y_dt2 = dde.grad.hessian(y, t)
    return  d2y_dt2 - mu * (1 - y ** 2) * dy_dt + y

# Time domain definition
geom = dde.geometry.TimeDomain(0, t_f)

# Boundary domain function - for time domain
def boundary_l(t, on_initial):
    return on_initial and np.isclose(t[0], 0)

# x1(0) = 0.5
bc_func1 = lambda inputs, outputs, X: outputs - X0[0]

# x2(0) = 0
bc_func2 = lambda inputs, outputs, X: dde.grad.jacobian(outputs, inputs, i=0, j=None) - X0[1]

# Defining as ICs
ic1 = dde.icbc.OperatorBC(geom, bc_func1, boundary_l)
ic2 = dde.icbc.OperatorBC(geom, bc_func2, boundary_l)

def train():
    with wandb.init() as run:
        config = wandb.config

        # Define the PDE problem
        data = dde.data.TimePDE(geom, VdP, [ic1, ic2], config["num_domain"], 2, num_test=50000)

        # Solver architecture
        layer_size = [1] + [config["height"]] * config["depth"] + [1]
        # layer_size = [1] + [10]*2 + [2]*2 + [10]*2 + [1]          # Fun experiment! Does there exist a linear coordinate frame for the ODE?
        activation = "tanh"
        initializer = "Glorot uniform"
        net = dde.nn.FNN(layer_size, activation, initializer)

        def input_transform(t):
            return tf.concat(
                (
                    t,
                    # t%1,
                    # tf.log_sigmoid(t),
                    tf.cos(t),
                    tf.sin(t)
                    # tf.sin(2 * t),
                    # tf.sin(3 * t),
                    # tf.sin(4 * t)
                ),
                axis=1,
            )

        net.apply_feature_transform(input_transform)


        # Create a Model using the PDE and neural network definitions
        model = dde.Model(data, net)

        # Compile the model!
        model.compile("L-BFGS")
        loss_history, train_state = model.train()
        
        loss_test = train_state.best_loss_test
        print(loss_test)
        
        wandb.log({"val_loss": loss_test})

# model.compile(
#     "adam", lr=alpha, loss_weights=loss_weights     # Removed metrics=["l2 relative error"]
# )

# losshistory, train_state = model.train(iterations=20000)
# dde.saveplot(losshistory, train_state, issave=True, isplot=True)

count = 100 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

In [ ]:
# t = np.linspace(0, t_f+100, 5000)
# sol = true_sol(t)
# x1, x2 = np.hsplit(true_sol(t), 2)

# # Plot the solution
# plt.figure(figsize=(30, 5))
# plt.plot(t, x1, color="black", label="x1_true")

# t = t.reshape(5000, 1)
# x1_pred = model.predict(t)

# plt.plot(t, x1_pred, color="red", linestyle="dashed", label="x1_pred")
# plt.legend()
# plt.show()